# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv


# Import API key
from api_keys import geoapify_key

C:\Users\jhaug\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,15.56,95,100,1.55,AU,1712009030
1,1,codrington,-38.2667,141.9667,14.95,85,75,5.66,AU,1712009031
2,2,tofol,5.3248,163.0078,29.03,74,75,5.14,FM,1712009031
3,3,waitangi,-43.9535,-176.5597,14.49,82,100,0.89,NZ,1712009031
4,4,klyuchi,52.2667,79.1667,3.21,70,100,3.79,RU,1712009031


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
     geo = True,
     tiles = "OSM",
     width = 1000,
     height = 500,
     color = "City",  # generates the side legend
     size = "Humidity"
)

# Display the map plot
# Note to grader: Must restart kernel to view more than one time; map is only good for one iteration....
map_plot_1

# Note: Leaving x- and y-axes as Longitude and Latitude, respectively, for presentation; not using x and y

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Looking for a place that is between 32 and 77 degrees Farenheit, humidity less than 50% 

narrow_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] > 0) & (city_data_df["Max Temp"] < 25) & (city_data_df["Humidity"] < 50)]
narrow_city_data_df = narrow_city_data_df.dropna(axis = 1)
narrow_city_data_df.reset_index(inplace=True)
#del narrow_city_data_df['index']
narrow_city_data_df.head()

# Drop any rows with null values
narrow_city_data_df.dropna(axis = 0) 

# Display sample data
narrow_city_data_df.head()

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,28,28,fredericton,45.9454,-66.6656,8.62,29,0,5.66,CA,1712008843
1,31,31,smithers,54.7804,-127.1743,7.33,39,100,3.60,CA,1712008837
2,36,36,villa canas,-34.0057,-61.6076,20.67,10,99,3.03,AR,1712009037
3,39,39,gaiman,-43.2897,-65.4929,18.54,15,3,3.28,AR,1712009038
4,49,49,badger,64.8000,-147.5333,4.43,38,39,0.38,US,1712009040


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\jhaug\AppData\Local\Temp\ipykernel_13548\1708204268.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,95,
1,codrington,AU,-38.2667,141.9667,85,
2,tofol,FM,5.3248,163.0078,74,
3,waitangi,NZ,-43.9535,-176.5597,82,
4,klyuchi,RU,52.2667,79.1667,70,
...,...,...,...,...,...,...
62,aden,YE,12.7794,45.0367,76,
63,lebu,CL,-37.6167,-73.6500,71,
64,nova sintra,CV,14.8667,-24.7167,80,
65,korla,CN,41.7597,86.1469,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 1  # finding the first hotel 
params = {
    #"limit":limit,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    params["categories"] = f'accommodation.hotel'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: Villa Howden
codrington - nearest hotel: No hotel found
tofol - nearest hotel: Nautilus Resort
waitangi - nearest hotel: Hotel Chathams
klyuchi - nearest hotel: No hotel found
trenton - nearest hotel: Lafayette Park Hotel
iqaluit - nearest hotel: Frobisher Inn
adamstown - nearest hotel: No hotel found
barretos - nearest hotel: Hotel Rodeio
margaret river - nearest hotel: Margaret River Hotel
albany - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
gadzhiyevo - nearest hotel: Оазис
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
anadyr - nearest hotel: Гостевой дом
kerema - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
toliara - nearest hotel: Ambary
vila velha - nearest hotel: Hotel Prainha
port-aux-francais - nearest hotel: Keravel
nassau - nearest hotel: Hilton Garden Inn
xiaoweizhai - nearest hotel: No hotel found
hamilton - nearest hotel: North 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,95,Villa Howden
1,codrington,AU,-38.2667,141.9667,85,No hotel found
2,tofol,FM,5.3248,163.0078,74,Nautilus Resort
3,waitangi,NZ,-43.9535,-176.5597,82,Hotel Chathams
4,klyuchi,RU,52.2667,79.1667,70,No hotel found
...,...,...,...,...,...,...
62,aden,YE,12.7794,45.0367,76,فندق قصر الجزيرة
63,lebu,CL,-37.6167,-73.6500,71,No hotel found
64,nova sintra,CV,14.8667,-24.7167,80,Residência Ka Dencho
65,korla,CN,41.7597,86.1469,71,Silver Star Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
# Configure the map plot
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
     geo = True,
     tiles = "OSM",
     width = 1000,
     height = 500,
     color= "City",  # generates the side legend
     size = "Humidity",   # maintaining size of points/markers
     hover_cols = ["Hotel Name","Country"]
)

# Display the map
map_plot_2

# Note: Leaving x- and y-axes as Longitude and Latitude, respectively, for presentation; not using x and y

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)